## Data Extraction

In [57]:
import pandas as pd
from sqlalchemy import create_engine, text
import psycopg2

In [58]:
ziiways_df = pd.read_csv("dataset/raw/ziiways_data.csv")

ziiways_df.head()


,Transaction_ID,Date,Customer_ID,Product_ID,Quantity,Unit_Price,Total_Cost,Discount_Rate,Sales_Channel,Order_Priority,...,Return_Reason,Payment_Type,Taxable,Region,Country,Customer_Name,Customer_Phone,Customer_Email,Customer_Address,Product_List_Title
0,200,2020-01-01 20:32:25.945945945,1086,536,3,120.436821,8265.374549,0.20,Online,High,...,Wrong Item,Wire Transfer,False,West,Canada,Customer 200,+1-652-572-9306,customer.200.78@example.com,"275 Second St, Phoenix, USA",Product 53
1,321,2020-01-02 06:55:08.108108108,1078,523,6,475.724994,4047.850479,NaN,Reseller,Critical,...,Damaged,PayPal,True,South,Mexico,Customer 321,+1-311-186-5760,customer.321.13@sample.com,"478 Third St, New York, USA",Product 33
2,989,2020-01-06 08:12:58.378378378,1077,535,3,146.400556,NaN,0.05,Direct,Critical,...,Damaged,PayPal,True,West,Canada,Customer 989,+1-922-606-9032,customer.989.99@example.com,"843 Second St, Phoenix, USA",Product 6
3,682,2020-01-07 22:03:14.594594594,1027,546,6,19.373194,8194.281993,NaN,Reseller,Medium,...,Wrong Item,Cash,True,South,Canada,Customer 682,+1-237-853-5808,customer.682.66@demo.com,"153 Main St, Phoenix, USA",Product 68
4,484,2020-01-07 07:08:06.486486486,1052,556,8,193.221313,8331.329249,0.20,Direct,Low,...,Late,Cash,False,South,Mexico,Customer 484,+1-986-360-9109,customer.484.3@sample.com,"264 Second St, New York, USA",Product 89


In [59]:
ziiways_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Transaction_ID         1005 non-null   int64  
 1   Date                   1005 non-null   object 
 2   Customer_ID            1005 non-null   int64  
 3   Product_ID             1005 non-null   int64  
 4   Quantity               1005 non-null   int64  
 5   Unit_Price             904 non-null    float64
 6   Total_Cost             905 non-null    float64
 7   Discount_Rate          714 non-null    float64
 8   Sales_Channel          1005 non-null   object 
 9   Order_Priority         1005 non-null   object 
 10  Warehouse_Code         1005 non-null   object 
 11  Ship_Mode              1005 non-null   object 
 12  Delivery_Status        1005 non-null   object 
 13  Customer_Satisfaction  1005 non-null   object 
 14  Item_Returned          1005 non-null   bool   
 15  Retu

In [60]:
ziiways_df.columns

Index(['Transaction_ID', 'Date', 'Customer_ID', 'Product_ID', 'Quantity',
       'Unit_Price', 'Total_Cost', 'Discount_Rate', 'Sales_Channel',
       'Order_Priority', 'Warehouse_Code', 'Ship_Mode', 'Delivery_Status',
       'Customer_Satisfaction', 'Item_Returned', 'Return_Reason',
       'Payment_Type', 'Taxable', 'Region', 'Country', 'Customer_Name',
       'Customer_Phone', 'Customer_Email', 'Customer_Address',
       'Product_List_Title'],
      dtype='object')

## Data Cleaning and Transformation

In [66]:
ziiways_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Transaction_ID         1005 non-null   int64         
 1   Date                   1005 non-null   datetime64[ns]
 2   Customer_ID            1005 non-null   int64         
 3   Product_ID             1005 non-null   int64         
 4   Quantity               1005 non-null   int64         
 5   Unit_Price             904 non-null    float64       
 6   Total_Cost             905 non-null    float64       
 7   Discount_Rate          714 non-null    float64       
 8   Sales_Channel          1005 non-null   object        
 9   Order_Priority         1005 non-null   object        
 10  Warehouse_Code         1005 non-null   object        
 11  Ship_Mode              1005 non-null   object        
 12  Delivery_Status        1005 non-null   object        
 13  Cus

In [65]:
# Converting the date datatype
ziiways_df["Date"] = pd.to_datetime(ziiways_df["Date"])

In [67]:
ziiways_cleaned_df =  ziiways_df.fillna({
    "Unit_Price": ziiways_df["Unit_Price"].mean(),
    "Total_Cost": ziiways_df["Total_Cost"].mean(),
    "Discount_Rate": 0,
    "Return_Reason": "Unknown"
})

ziiways_cleaned_df.head()

,Transaction_ID,Date,Customer_ID,Product_ID,Quantity,Unit_Price,Total_Cost,Discount_Rate,Sales_Channel,Order_Priority,...,Return_Reason,Payment_Type,Taxable,Region,Country,Customer_Name,Customer_Phone,Customer_Email,Customer_Address,Product_List_Title
0,200,2020-01-01 20:32:25.945945945,1086,536,3,120.436821,8265.374549,0.20,Online,High,...,Wrong Item,Wire Transfer,False,West,Canada,Customer 200,+1-652-572-9306,customer.200.78@example.com,"275 Second St, Phoenix, USA",Product 53
1,321,2020-01-02 06:55:08.108108108,1078,523,6,475.724994,4047.850479,0.00,Reseller,Critical,...,Damaged,PayPal,True,South,Mexico,Customer 321,+1-311-186-5760,customer.321.13@sample.com,"478 Third St, New York, USA",Product 33
2,989,2020-01-06 08:12:58.378378378,1077,535,3,146.400556,5096.553818,0.05,Direct,Critical,...,Damaged,PayPal,True,West,Canada,Customer 989,+1-922-606-9032,customer.989.99@example.com,"843 Second St, Phoenix, USA",Product 6
3,682,2020-01-07 22:03:14.594594594,1027,546,6,19.373194,8194.281993,0.00,Reseller,Medium,...,Wrong Item,Cash,True,South,Canada,Customer 682,+1-237-853-5808,customer.682.66@demo.com,"153 Main St, Phoenix, USA",Product 68
4,484,2020-01-07 07:08:06.486486486,1052,556,8,193.221313,8331.329249,0.20,Direct,Low,...,Late,Cash,False,South,Mexico,Customer 484,+1-986-360-9109,customer.484.3@sample.com,"264 Second St, New York, USA",Product 89


In [68]:
ziiways_cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Transaction_ID         1005 non-null   int64         
 1   Date                   1005 non-null   datetime64[ns]
 2   Customer_ID            1005 non-null   int64         
 3   Product_ID             1005 non-null   int64         
 4   Quantity               1005 non-null   int64         
 5   Unit_Price             1005 non-null   float64       
 6   Total_Cost             1005 non-null   float64       
 7   Discount_Rate          1005 non-null   float64       
 8   Sales_Channel          1005 non-null   object        
 9   Order_Priority         1005 non-null   object        
 10  Warehouse_Code         1005 non-null   object        
 11  Ship_Mode              1005 non-null   object        
 12  Delivery_Status        1005 non-null   object        
 13  Cus

In [69]:
ziiways_cleaned_df.columns

Index(['Transaction_ID', 'Date', 'Customer_ID', 'Product_ID', 'Quantity',
       'Unit_Price', 'Total_Cost', 'Discount_Rate', 'Sales_Channel',
       'Order_Priority', 'Warehouse_Code', 'Ship_Mode', 'Delivery_Status',
       'Customer_Satisfaction', 'Item_Returned', 'Return_Reason',
       'Payment_Type', 'Taxable', 'Region', 'Country', 'Customer_Name',
       'Customer_Phone', 'Customer_Email', 'Customer_Address',
       'Product_List_Title'],
      dtype='object')

In [70]:
products_dim = ziiways_cleaned_df[["Product_ID","Product_List_Title"]].copy().drop_duplicates().reset_index(drop=True)

products_dim

,Product_ID,Product_List_Title
0,536,Product 53
1,523,Product 33
2,535,Product 6
3,546,Product 68
4,556,Product 89
...,...,...
917,533,Product 16
918,542,Product 38
919,511,Product 48
920,504,Product 38


In [71]:
locations_dim = ziiways_cleaned_df[["Region", "Country"]].copy().drop_duplicates().reset_index(drop=True)

locations_dim.index.name = "location_id"
locations_dim = locations_dim.reset_index()
locations_dim["location_id"] += 1

locations_dim

,location_id,Region,Country
0,1,West,Canada
1,2,South,Mexico
2,3,South,Canada
3,4,South,USA
4,5,North,Canada
5,6,East,Mexico
6,7,East,Canada
7,8,North,Mexico
8,9,North,USA
9,10,West,USA


In [73]:
customers_dim = ziiways_cleaned_df[['Customer_ID', 'Customer_Name', 'Customer_Phone', 'Customer_Email', 'Customer_Address', 'Region', 'Country']].copy().drop_duplicates().reset_index(drop=True)

# customers_dim.index.name = "customer_id"
# customers_dim = customers_dim.reset_index()
# customers_dim["customer_id"] += 1


# Merge customers_dim with location to derive "billing_location_id"
customers_dim = customers_dim.merge(locations_dim, on=['Region', 'Country'], how="left")
customers_dim = customers_dim[['Customer_ID', 'Customer_Name', 'Customer_Phone', 'Customer_Email', 'Customer_Address', 'location_id']]

customers_dim = customers_dim.rename(columns={"location_id": "billing_location_id"})

customers_dim

,Customer_ID,Customer_Name,Customer_Phone,Customer_Email,Customer_Address,billing_location_id
0,1086,Customer 200,+1-652-572-9306,customer.200.78@example.com,"275 Second St, Phoenix, USA",1
1,1078,Customer 321,+1-311-186-5760,customer.321.13@sample.com,"478 Third St, New York, USA",2
2,1077,Customer 989,+1-922-606-9032,customer.989.99@example.com,"843 Second St, Phoenix, USA",1
3,1027,Customer 682,+1-237-853-5808,customer.682.66@demo.com,"153 Main St, Phoenix, USA",3
4,1052,Customer 484,+1-986-360-9109,customer.484.3@sample.com,"264 Second St, New York, USA",2
...,...,...,...,...,...,...
995,1095,Customer 250,+1-875-504-2878,customer.250.26@sample.com,"664 Main St, Chicago, USA",3
996,1008,Customer 412,+1-645-470-5077,customer.412.80@sample.com,"699 Third St, Houston, USA",4
997,1060,Customer 816,+1-162-396-1771,customer.816.55@sample.com,"453 Fourth St, Los Angeles, USA",11
998,1054,Customer 702,+1-130-713-1098,customer.702.30@sample.com,"438 Fourth St, Houston, USA",1


In [78]:



transactions_fact = ziiways_cleaned_df.merge(customers_dim, on=['Customer_ID','Customer_Name', 'Customer_Phone', 'Customer_Email', 'Customer_Address'], how="left") \
                                     .merge(products_dim, on=['Product_ID', "Product_List_Title"], how="left") \
                                     .merge(locations_dim, on= ["Region", "Country"], how="left") \
                                     .rename(columns={"location_id": "shipping_location_id"}) 
                                     
    
transactions_fact = transactions_fact[['Transaction_ID', 'Date', 'Customer_ID', 'Product_ID', 'Quantity',
       'Unit_Price', 'Total_Cost', 'Discount_Rate', 'Sales_Channel',
       'Order_Priority', 'Warehouse_Code', 'Ship_Mode', 'Delivery_Status',
       'Customer_Satisfaction', 'Item_Returned', 'Return_Reason',
       'Payment_Type', 'Taxable', 'shipping_location_id']]

# transactions_fact.index.name = "transaction_id"
# transactions_fact = transactions_fact.reset_index()
# transactions_fact["transaction_id"] += 1
                    

transactions_fact

,Transaction_ID,Date,Customer_ID,Product_ID,Quantity,Unit_Price,Total_Cost,Discount_Rate,Sales_Channel,Order_Priority,Warehouse_Code,Ship_Mode,Delivery_Status,Customer_Satisfaction,Item_Returned,Return_Reason,Payment_Type,Taxable,shipping_location_id
0,200,2020-01-01 20:32:25.945945945,1086,536,3,120.436821,8265.374549,0.20,Online,High,WH-3,2-Day,Cancelled,Neutral,False,Wrong Item,Wire Transfer,False,1
1,321,2020-01-02 06:55:08.108108108,1078,523,6,475.724994,4047.850479,0.00,Reseller,Critical,WH-1,Overnight,Backorder,Satisfied,True,Damaged,PayPal,True,2
2,989,2020-01-06 08:12:58.378378378,1077,535,3,146.400556,5096.553818,0.05,Direct,Critical,WH-1,Overnight,Pending,Unsatisfied,True,Damaged,PayPal,True,1
3,682,2020-01-07 22:03:14.594594594,1027,546,6,19.373194,8194.281993,0.00,Reseller,Medium,WH-1,Express,Pending,Unsatisfied,False,Wrong Item,Cash,True,3
4,484,2020-01-07 07:08:06.486486486,1052,556,8,193.221313,8331.329249,0.20,Direct,Low,WH-2,2-Day,Delivered,Satisfied,True,Late,Cash,False,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000,250,2020-01-05 02:22:42.162162162,1095,533,5,490.760429,1138.724711,0.00,Reseller,Medium,WH-3,Express,Delivered,Unsatisfied,True,Damaged,Credit Card,True,3
1001,412,2020-01-07 11:27:34.054054054,1008,542,9,236.240910,1786.865600,0.10,Online,Medium,WH-3,2-Day,Cancelled,Neutral,True,Not Specified,Cash,True,4
1002,816,2020-01-06 02:48:38.918918918,1060,511,6,256.467791,6443.682500,0.20,Partner,Medium,WH-3,2-Day,Cancelled,Unsatisfied,True,Not Specified,Cash,False,11
1003,702,2020-01-09 08:38:55.135135135,1054,504,6,256.467791,4814.744796,0.00,Online,Medium,WH-2,Regular,Cancelled,Satisfied,False,Damaged,Cash,True,1


In [44]:
ziiways_cleaned_df.columns

Index(['Transaction_ID', 'Date', 'Customer_ID', 'Product_ID', 'Quantity',
       'Unit_Price', 'Total_Cost', 'Discount_Rate', 'Sales_Channel',
       'Order_Priority', 'Warehouse_Code', 'Ship_Mode', 'Delivery_Status',
       'Customer_Satisfaction', 'Item_Returned', 'Return_Reason',
       'Payment_Type', 'Taxable', 'Region', 'Country', 'Customer_Name',
       'Customer_Phone', 'Customer_Email', 'Customer_Address',
       'Product_List_Title'],
      dtype='object')

In [79]:
# Saving dataframes into CSVs

# Saving Cleaned Data
ziiways_cleaned_df.to_csv("dataset/interim/ziiways_cleaned.csv", index=False)

# Saving Dim and Fact Tables
customers_dim.to_csv("dataset/processed/customers_dim.csv", index=False)
products_dim.to_csv("dataset/processed/products_dim.csv", index=False)
locations_dim.to_csv("dataset/processed/locations_dim.csv", index=False)
transactions_fact.to_csv("dataset/processed/transactions_fact.csv", index=False)